This file consists of all feature selection methods that were tried and evaluated that aren't used in the final model

In [ ]:
## Mutual information

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Select Features using Mutual Information
mi_scores = mutual_info_classif(X_train_scaled, y_train)
mi_feature_indices = np.argsort(mi_scores)[-2100:]  # Select top 100 features
X_train_mi = X_train_scaled[:, mi_feature_indices]
X_val_mi = X_val_scaled[:, mi_feature_indices]

# Train Random Forest with Selected Features
rf_mi = RandomForestClassifier(random_state=42)
rf_mi.fit(X_train_mi, y_train)
y_pred_mi = rf_mi.predict(X_val_mi)

# Compute Accuracy, classification report & Confusion Matrix
accuracy_mi = accuracy_score(y_val, y_pred_mi)
print(classification_report(y_val, y_pred_mi))
cm_mi = confusion_matrix(y_val, y_pred_mi)

print(f"Accuracy (Mutual Information Feature Selection): {accuracy_mi:.4f}")

# Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm_mi, annot=True, fmt='d', cmap='Blues', xticklabels=["Normal", "Abnormal"], yticklabels=["Normal", "Abnormal"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()